In [56]:
import pandas as pd
import numpy as np
import re

In [57]:
facture = pd.read_csv("facture.csv", sep="|") 
facture

,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,12345_GEO (0001BB),1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,12345_GEO (0001BB),1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,12345_GEO (0001BB),1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,12345_GEO (0001BB),1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,12345_GEO (0001BB),1300082549,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...,...
51427,19767_GEO (0632KO),FV1/055100030,30/08/2019,31/12/2019,3 028,"397,18"
51428,19767_GEO (0632KO),055100058,01/01/2020,31/03/2020,2 163,"283,57"
51429,19767_GEO (0632KO),055100113,01/04/2020,30/06/2020,2 160,"312,55"
51430,19767_GEO (0632KO),055100168,01/07/2020,30/09/2020,2 230,"294,07"


## 1) Le nom d'une antenne a un format bien précis : il est toujours composé de 4 chiffres et de deux lettres. On te demande de retirer tout autre caractère de la colonne "SITE_NAME".

In [58]:
facture['SITE_NAME'] = facture['SITE_NAME'].astype(str).str.split().str[1]
facture

,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,(0001BB),1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,(0001BB),1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,(0001BB),1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,(0001BB),1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,(0001BB),1300082549,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...,...
51427,(0632KO),FV1/055100030,30/08/2019,31/12/2019,3 028,"397,18"
51428,(0632KO),055100058,01/01/2020,31/03/2020,2 163,"283,57"
51429,(0632KO),055100113,01/04/2020,30/06/2020,2 160,"312,55"
51430,(0632KO),055100168,01/07/2020,30/09/2020,2 230,"294,07"


## 2) On te demande de supprimer la colonne "INVOICE_NAME" qui n'apporte rien aux équipes

In [43]:
del facture["INVOICE_NAME"]
facture

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,(0001BB),01/01/2018,31/01/2018,2 447,"316,46"
1,(0001BB),01/02/2018,28/02/2018,2 211,"286,82"
2,(0001BB),01/03/2018,31/03/2018,2 450,"317,32"
3,(0001BB),01/04/2018,30/04/2018,2 431,"315,00"
4,(0001BB),01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...
51427,(0632KO),30/08/2019,31/12/2019,3 028,"397,18"
51428,(0632KO),01/01/2020,31/03/2020,2 163,"283,57"
51429,(0632KO),01/04/2020,30/06/2020,2 160,"312,55"
51430,(0632KO),01/07/2020,30/09/2020,2 230,"294,07"


## 3) Il y a des antennes pour lesquelles on a des factures mensuelles tandis que pour d'autres, on les reçoit moins régulièrement (trimestre, semestre, année). Ajoute une colonne à ton tableau qui calcule le nombre de jours entre le début de la période de facturation et la fin de la période de facturation.

In [44]:
facture[['BEGIN','END']] = facture[['BEGIN','END']].apply(pd.to_datetime)
facture['TIME_PERIOD'] = (facture['END'] - facture['BEGIN']).dt.days
facture

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,TIME_PERIOD
0,(0001BB),2018-01-01,2018-01-31,2 447,"316,46",30
1,(0001BB),2018-01-02,2018-02-28,2 211,"286,82",57
2,(0001BB),2018-01-03,2018-03-31,2 450,"317,32",87
3,(0001BB),2018-01-04,2018-04-30,2 431,"315,00",116
4,(0001BB),2018-01-05,2018-05-31,2 574,"333,37",146
...,...,...,...,...,...,...
51427,(0632KO),2019-08-30,2019-12-31,3 028,"397,18",123
51428,(0632KO),2020-01-01,2020-03-31,2 163,"283,57",90
51429,(0632KO),2020-01-04,2020-06-30,2 160,"312,55",178
51430,(0632KO),2020-01-07,2020-09-30,2 230,"294,07",267


## 4) À partir de cette nouvelle colonne, crée une autre colonne qui nous donne la consommation journalière moyenne de la période.

In [45]:
facture['KWH_PERIOD'] = facture['KWH_PERIOD'].str.replace(" ","")
facture['KWH_DAY'] = (facture['KWH_PERIOD'].astype(int) / facture['TIME_PERIOD'].astype(int))
facture = facture[['SITE_NAME', 'BEGIN', 'END', 'TIME_PERIOD', 'KWH_PERIOD', 'KWH_DAY', 'PERIOD_EUR']]
facture

,SITE_NAME,BEGIN,END,TIME_PERIOD,KWH_PERIOD,KWH_DAY,PERIOD_EUR
0,(0001BB),2018-01-01,2018-01-31,30,2447,81.566667,"316,46"
1,(0001BB),2018-01-02,2018-02-28,57,2211,38.789474,"286,82"
2,(0001BB),2018-01-03,2018-03-31,87,2450,28.160920,"317,32"
3,(0001BB),2018-01-04,2018-04-30,116,2431,20.956897,"315,00"
4,(0001BB),2018-01-05,2018-05-31,146,2574,17.630137,"333,37"
...,...,...,...,...,...,...,...
51427,(0632KO),2019-08-30,2019-12-31,123,3028,24.617886,"397,18"
51428,(0632KO),2020-01-01,2020-03-31,90,2163,24.033333,"283,57"
51429,(0632KO),2020-01-04,2020-06-30,178,2160,12.134831,"312,55"
51430,(0632KO),2020-01-07,2020-09-30,267,2230,8.352060,"294,07"


## 5) Merci d'ajouter également une colonne qui donne, pour chaque antenne, le nombre de factures dont on dispose au total. Pas grave si l'information se répète à chaque ligne pour une même antenne.

In [59]:
facture['FACT_NBR'] = facture.groupby('SITE_NAME')['SITE_NAME'].transform('count')
facture.to_csv('new_facture.csv') 

## 6) Et information bonus si tu y arrives : les équipes aimeraient beaucoup avoir une liste de toutes les antennes avec, pour chaque antenne, le coefficient de variation de la consommation. Cela permettra d'avoir une idée, pour chaque antenne, de la dispersion de sa consommation d'énergie.

In [47]:
#Coefficient de variation = (Écart-type / Moyenne) * 100.
#écart-type = écart-type de la consommation (écart moyen entre la consommation effective et la consommation moyenne) = .std()

newfacture = facture[['SITE_NAME', 'KWH_DAY']].copy()
newfacture['KWH_DAY'].astype('float64')
newfacture['MEAN'] = newfacture.groupby('SITE_NAME')['KWH_DAY'].transform('mean')
newfacture.std()

/var/folders/35/cnwwxl_x001czjgx_rqp6b4h0000gq/T/ipykernel_5569/3935137280.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  newfacture.std()


KWH_DAY           NaN
MEAN       156.429578
dtype: float64

## 7) Enfin, tu dois exporter ces deux fichiers obtenus pour pouvoir l'envoyer aux équipes de l'opérateur et leur demander si c'est bien cela qu'ils attendaient de toi. 